In [1]:
import sys
sys.path.insert(1, '/home/b3arjuden/crocket')

from datetime import datetime
from decimal import Decimal
import itertools
from math import floor
from numpy import exp, linspace, median, mean, std
from pprint import pprint
from os import environ
from os.path import join
from scipy.stats import mannwhitneyu, ttest_ind, variation


import matplotlib.pyplot as plt
%matplotlib inline

from crocket.sql.sql import Database
from crocket.utilities.credentials import get_credentials

In [2]:
# Environment variables

HOME_DIRECTORY_PATH = environ['HOME']
CREDENTIALS_FILE_PATH = join(HOME_DIRECTORY_PATH, '.credentials_unlocked.json')

HOSTNAME = 'localhost'
DATABASE_NAME = 'PRODUCTION40'

In [3]:
# Load SQL username and password
USERNAME, PASSCODE = get_credentials(CREDENTIALS_FILE_PATH)

# Create database connection
db = Database(hostname=HOSTNAME,
              username=USERNAME,
              password=PASSCODE,
              database_name=DATABASE_NAME)

In [4]:
tables = db.get_all_tables()

In [13]:
def run_algorithm(data, status,
                  duration=1,
                  price_lag_time=30,
                  price_lag_duration=5,
                  price_lag_threshold=0.05,
                  volume_lag_duration=60,
                  buy_volume_lag_min=10,
                  buy_volume_lag_max=40,
                  sell_volume_lag_min=10,
                  sell_volume_lag_max=40,
                  profit_percent=0.04,
                  stop_loss_percent=0.01,
                  stop_gain_percent=0.02,
                  max_hold_time=14400,
                  wait_time=14400,
                  digits=Decimal('1e-8')):

    time = data.get('datetime')
    buyvolume = data.get('buy_volume')
    sellvolume = data.get('sell_volume')
    wprice = data.get('wprice')

    current_time = time[-1]
    current_price = wprice[-1]
#     print(current_price)
#     if current_price == 0.00247892:
#         print(current_time, current_price)
#         input()
        
    last_buy_time_difference = (current_time - status.get('last_buy').get('start')).total_seconds()

    # Action if haven't bought coin
    if not status.get('bought'):

        # No action if purchased within time of last buy
        if last_buy_time_difference < wait_time:
            return status

        sample_buy_volume_mean = mean(buyvolume[-duration:])
        buy_volume_lag_total = sum(buyvolume[-(duration + volume_lag_duration):-duration])
        sell_volume_lag_total = sum(sellvolume[-(duration + volume_lag_duration):-duration])

        if sample_buy_volume_mean > 0 and \
            buy_volume_lag_min < buy_volume_lag_total < buy_volume_lag_max and \
                sell_volume_lag_min < sell_volume_lag_total < sell_volume_lag_max:
                    
            previous_price = Decimal(
                mean(wprice[-(duration + price_lag_time):-(duration + price_lag_time - price_lag_duration)])).quantize(digits)

            if sample_buy_volume_mean > 2 and \
                    abs((current_price - previous_price) / previous_price) < price_lag_threshold:
                # TODO: MAKE API CALL TO BUY (wrap in try)
                status['bought'] = True
                status['buy_lag'] = True
                status['current_buy'] = {'start': current_time,
                                         'buy_price': current_price,
                                         'buy_volume': sample_buy_volume_mean,
                                         'last30_buyvolume': sum(buyvolume[-(30+duration):-duration]),
                                         'last60_buyvolume': sum(buyvolume[-(60+duration):-duration]),
                                         'last30_sellvolume': sum(sellvolume[-(30+duration):-duration]),
                                         'last60_sellvolume': sum(sellvolume[-(60+duration):-duration])}

    # Action if have bought coin
    else:
        if 'buy_lag' in status and status.get('buy_lag'):
            status['current_buy']['lag_buy_price'] = current_price
            status['buy_lag'] = False
        
        current_buy = status.get('current_buy')
        current_buy_hold_time = (current_time - current_buy.get('start')).total_seconds()

        stop_gain_threshold = (current_buy.get('buy_price') * Decimal(stop_gain_percent + 1)).quantize(digits)

        # Activate stop gain signal after passing threshold percentage
        if current_price > stop_gain_threshold:
            status['stop_gain'] = True

        # Activate maximize gain signal after passing profit threshold
        if current_price > (current_buy.get('buy_price') * Decimal(profit_percent + 1)).quantize(digits):
            status['maximize_gain'] = True

        # Sell if hit stop loss
        # Sell after hitting profit threshold followed by drop in price of X%
        # Sell after passing max hold time
        # Sell after detecting stop gain signal and price drop below stop gain price
        if (current_price < (current_buy.get('buy_price') * Decimal(1 - stop_loss_percent)).quantize(digits)) or \
                status.get('maximize_gain') or \
                current_buy_hold_time > max_hold_time or \
                (status.get('stop_gain') and current_price < stop_gain_threshold):
            # TODO: MAKE API CALL TO SELL (wrap in try)
            status['current_buy']['stop'] = current_time

            buy_price = current_buy.get('buy_price')

            status['current_buy']['sell_price'] = current_price
            status['current_buy']['profit'] = (((current_price - buy_price) / buy_price) * Decimal(0.995)).quantize(digits)

            status['bought'] = False
            status['stop_gain'] = False
            status['maximize_gain'] = False

            status['last_buy'] = {'start': current_buy.get('start'),
                                  'buy_price': buy_price}

    return status

In [24]:
def run_algorithm2(data, status,
                  duration=1,
                  price_lag_time=30,
                  price_lag_duration=5,
                  price_lag_threshold=0.05,
                  volume_lag_duration=60,
                  buy_volume_lag_min=10,
                  buy_volume_lag_max=40,
                  sell_volume_lag_min=10,
                  sell_volume_lag_max=40,
                  profit_percent=0.04,
                  stop_loss_percent=0.01,
                  stop_gain_percent=0.02,
                  stop_gain_increment=0.02,
                  max_hold_time=14400,
                  wait_time=14400,
                  digits=Decimal('1e-8')):

    time = data.get('datetime')
    buyvolume = data.get('buy_volume')
    sellvolume = data.get('sell_volume')
    wprice = data.get('wprice')

    current_time = time[-1]
    current_price = wprice[-1]
#     print(current_price)
#     if current_price == 0.00247892:
#         print(current_time, current_price)
#         input()
        
    last_buy_time_difference = (current_time - status.get('last_buy').get('start')).total_seconds()

    # Action if haven't bought coin
    if not status.get('bought'):

        # No action if purchased within time of last buy
        if last_buy_time_difference < wait_time:
            return status

        sample_buy_volume_mean = mean(buyvolume[-duration:])
        buy_volume_lag_total = sum(buyvolume[-(duration + volume_lag_duration):-duration])
        sell_volume_lag_total = sum(sellvolume[-(duration + volume_lag_duration):-duration])

        if sample_buy_volume_mean > 0 and \
            buy_volume_lag_min < buy_volume_lag_total < buy_volume_lag_max and \
                sell_volume_lag_min < sell_volume_lag_total < sell_volume_lag_max:
                    
            previous_price = Decimal(
                mean(wprice[-(duration + price_lag_time):-(duration + price_lag_time - price_lag_duration)])).quantize(digits)

            if sample_buy_volume_mean > 2 and \
                    abs((current_price - previous_price) / previous_price) < price_lag_threshold:
                # TODO: MAKE API CALL TO BUY (wrap in try)
                status['bought'] = True
                status['buy_lag'] = True
                status['current_buy'] = {'start': current_time,
                                         'buy_price': current_price,
                                         'buy_volume': sample_buy_volume_mean,
                                         'last30_buyvolume': sum(buyvolume[-(30+duration):-duration]),
                                         'last60_buyvolume': sum(buyvolume[-(60+duration):-duration]),
                                         'last30_sellvolume': sum(sellvolume[-(30+duration):-duration]),
                                         'last60_sellvolume': sum(sellvolume[-(60+duration):-duration])}

    # Action if have bought coin
    else:
        if 'buy_lag' in status and status.get('buy_lag'):
            status['current_buy']['lag_buy_price'] = current_price
            status['buy_lag'] = False
        
        current_buy = status.get('current_buy')
        current_buy_hold_time = (current_time - current_buy.get('start')).total_seconds()

        if status.get('current_stop_gain_percent') == 0.02:
            loss_threshold = 0
        else:
            loss_threshold = 0.01
            
        current_stop_gain_threshold = (current_buy.get('buy_price') * Decimal(status.get('current_stop_gain_percent') + 1)).quantize(digits)
        current_stop_gain_min_threshold = (current_buy.get('buy_price') * Decimal(status.get('current_stop_gain_percent') - loss_threshold + 1)).quantize(digits)
        
        next_stop_gain_threshold = (current_buy.get('buy_price') * Decimal(status.get('current_stop_gain_percent') + stop_gain_increment + 1)).quantize(digits)
        
        # Activate stop gain signal after passing threshold percentage
        if not status.get('stop_gain') and current_price > current_stop_gain_threshold:
            status['stop_gain'] = True
        elif status.get('stop_gain') and current_price > next_stop_gain_threshold:
            status['current_stop_gain_percent'] = status.get('current_stop_gain_percent') + stop_gain_increment

        # Sell if hit stop loss
        # Sell after hitting profit threshold followed by drop in price of X%
        # Sell after passing max hold time
        # Sell after detecting stop gain signal and price drop below stop gain price
        if (current_price < (current_buy.get('buy_price') * Decimal(1 - stop_loss_percent)).quantize(digits)) or \
                status.get('maximize_gain') or \
                current_buy_hold_time > max_hold_time or \
                (status.get('stop_gain') and current_price < current_stop_gain_min_threshold):
            # TODO: MAKE API CALL TO SELL (wrap in try)
            status['current_buy']['stop'] = current_time

            buy_price = current_buy.get('buy_price')

            status['current_buy']['sell_price'] = current_price
            status['current_buy']['profit'] = (((current_price - buy_price) / buy_price) * Decimal(0.995)).quantize(digits)

            status['bought'] = False
            status['stop_gain'] = False
            status['maximize_gain'] = False

            status['last_buy'] = {'start': current_buy.get('start'),
                                  'buy_price': buy_price}

    return status

In [25]:
def benchmark():
    
    markets = db.get_all_tables()
    #markets = ['BTC-NEO']
    running_data = {}
    status = {}
    all_calls = {}
    
    bought_time = datetime(2017, 11, 11, 11, 11).astimezone(tz=None)

    last_buy = {'start': bought_time,
                'buy_price': 0}
    
    total_profit = 0
    total_lag_profit = 0
    
    for market in markets:
        status[market] = {'bought': False,
                          'last_buy': last_buy,
                          'current_buy': {},
                          'stop_gain': False,
                          'current_stop_gain_percent': 0.02}

        running_data[market] = {'datetime': [],
                        'wprice': [],
                        'buy_volume': [],
                        'sell_volume': []}
        
        all_calls[market] = []
        
    for market in markets:
        
        if market not in skip_list:
            data = db.select_query(market, '*')
            
            if data:
                time, price, wprice, volume, buyvolume, sellvolume, buyorder, sellorder = zip(*data)
                
                time = [x.astimezone(tz=None) for x in time]
                
                for ind in range(65, len(volume)):
                    running_data[market]['datetime'] = time[ind-65:ind]
                    running_data[market]['wprice'] = wprice[ind-65:ind]
                    running_data[market]['buy_volume'] = buyvolume[ind-65:ind]
                    running_data[market]['sell_volume'] = sellvolume[ind-65:ind]
                    
                    status[market] = run_algorithm2(running_data.get(market), status.get(market))
                    
                    completed_buy = status.get(market).get('current_buy')
                    
                    if completed_buy.get('profit'):
                        
                        completed_buy['lag_sell_price'] = wprice[ind]
                        completed_buy['lag_profit'] = (((completed_buy.get('lag_sell_price') - completed_buy.get('lag_buy_price')) / completed_buy.get('lag_buy_price')) * Decimal(0.995)).quantize(Decimal(10) ** -8)
                        
                        status[market]['current_buy'] = {}
                        all_calls[market].append(completed_buy)
                        
                        
                if all_calls.get(market):
                    profit = sum([x.get('profit') for x in all_calls.get(market) if 'profit' in x])
                    lag_profit = sum([x.get('lag_profit') for x in all_calls.get(market) if 'lag_profit' in x])
                    print('For {}, profit: {}, lag profit: {}'.format(market, str(profit), str(lag_profit)))
                    
                    total_profit += profit
                    total_lag_profit += lag_profit
    
    print('Total profit: {}'.format(total_profit))
    print('Total lag profit: {}'.format(total_lag_profit))
    
    return all_calls

In [26]:
#skip_list = ['BTC-BCC', 'BTC-ETH', 'BTC-LSK', 'BTC-NEO', 'BTC-OMG', 'BTC-XRP', 'BTC-LTC']
skip_list = []

calls = benchmark()

For BTC-ADA, profit: 0.17671329, lag profit: 0.18826413
For BTC-ADX, profit: -0.03168720, lag profit: -0.02449497
For BTC-AEON, profit: -0.02022561, lag profit: 0.01010984
For BTC-ARDR, profit: -0.10510427, lag profit: -0.12904478
For BTC-BAT, profit: 0.01833088, lag profit: 0.00799489
For BTC-BAY, profit: 0.01722115, lag profit: 0.01715517
For BTC-BCC, profit: -0.02332203, lag profit: -0.01661429
For BTC-BTG, profit: 0.01992864, lag profit: 0.01875633
For BTC-CVC, profit: 0.15112572, lag profit: 0.13977485
For BTC-DASH, profit: 0.14880135, lag profit: 0.14118064
For BTC-DCR, profit: -0.01388836, lag profit: -0.01709418
For BTC-DGB, profit: -0.06091153, lag profit: -0.06707867
For BTC-EDG, profit: -0.05751890, lag profit: -0.06712279
For BTC-EMC2, profit: 0.35485058, lag profit: 0.32074921
For BTC-ETC, profit: 0.10182960, lag profit: 0.10317028
For BTC-FCT, profit: -0.02152184, lag profit: -0.03251613
For BTC-FUN, profit: -0.02926471, lag profit: 0E-8
For BTC-GBYTE, profit: 0.01849560,

In [22]:
pos_calls = []
neg_calls = []

for k in calls.keys():
    
    for c in calls.get(k):
        
        if c.get('lag_profit') > 0.05:
            
            pos_calls.append(c)
        
        else:
            
            neg_calls.append(c)

In [27]:
total_profit = []
all_calls = []

for k in calls.keys():
    
    for x in calls.get(k):
        
        if x.get('last60_buyvolume') > 10 and x.get('last60_buyvolume') < 40 and \
        x.get('last60_sellvolume') > 10 and x.get('last60_sellvolume') < 40:
            x['market'] = k
            total_profit.append(x.get('profit'))
            all_calls.append(x)
        
print('Total profit: {}'.format(str(sum(total_profit))))
print('Number of calls: {}'.format(str(len(total_profit))))

Total profit: 2.60791378
Number of calls: 348


In [9]:
all_calls

[{'buy_price': Decimal('0.00000778'),
  'buy_volume': Decimal('6.00772065'),
  'lag_buy_price': Decimal('0.00000778'),
  'lag_profit': Decimal('0.01406812'),
  'lag_sell_price': Decimal('0.00000789'),
  'last30_buyvolume': Decimal('14.00598844'),
  'last30_sellvolume': Decimal('8.69550901'),
  'last60_buyvolume': Decimal('28.37464022'),
  'last60_sellvolume': Decimal('20.64332299'),
  'market': 'BTC-ADA',
  'profit': Decimal('0.01151028'),
  'sell_price': Decimal('0.00000787'),
  'start': datetime.datetime(2017, 12, 9, 21, 51, 40, tzinfo=datetime.timezone(datetime.timedelta(-1, 57600), 'PST')),
  'stop': datetime.datetime(2017, 12, 9, 22, 1, 40, tzinfo=datetime.timezone(datetime.timedelta(-1, 57600), 'PST'))},
 {'buy_price': Decimal('0.00000765'),
  'buy_volume': Decimal('4.05626074'),
  'lag_buy_price': Decimal('0.00000769'),
  'lag_profit': Decimal('0.01811443'),
  'lag_sell_price': Decimal('0.00000783'),
  'last30_buyvolume': Decimal('12.83458650'),
  'last30_sellvolume': Decimal('2

In [ ]:
start = []
stop = []

for c in sorted_calls:
    
    start.append(int((c[2] - sorted_calls[0][2]).total_seconds()))
    stop.append(int((c[3] - sorted_calls[0][2]).total_seconds()))

In [9]:
sorted_calls = sorted([(x.get('market'), x.get('buy_price'), x.get('sell_price'), x.get('profit'), x.get('start'), x.get('stop')) for x in all_calls], key=lambda x:x[4])

In [10]:
sorted_calls

[('BTC-DASH',
  Decimal('0.04899006'),
  Decimal('0.04834229'),
  Decimal('-0.01315637'),
  datetime.datetime(2017, 12, 9, 19, 10, 40, tzinfo=datetime.timezone(datetime.timedelta(-1, 57600), 'PST')),
  datetime.datetime(2017, 12, 9, 19, 19, 40, tzinfo=datetime.timezone(datetime.timedelta(-1, 57600), 'PST'))),
 ('BTC-NEO',
  Decimal('0.00247892'),
  Decimal('0.00245000'),
  Decimal('-0.01160804'),
  datetime.datetime(2017, 12, 9, 20, 4, 40, tzinfo=datetime.timezone(datetime.timedelta(-1, 57600), 'PST')),
  datetime.datetime(2017, 12, 9, 20, 7, 40, tzinfo=datetime.timezone(datetime.timedelta(-1, 57600), 'PST'))),
 ('BTC-SALT',
  Decimal('0.00054483'),
  Decimal('0.00053236'),
  Decimal('-0.02277343'),
  datetime.datetime(2017, 12, 9, 20, 5, 40, tzinfo=datetime.timezone(datetime.timedelta(-1, 57600), 'PST')),
  datetime.datetime(2017, 12, 9, 20, 6, 40, tzinfo=datetime.timezone(datetime.timedelta(-1, 57600), 'PST'))),
 ('BTC-XMR',
  Decimal('0.01689929'),
  Decimal('0.01669032'),
  Decimal

In [ ]:
field = 'last60_sellvolume'
x1 = [float(x.get(field)) for x in pos_calls]
x2 = [float(x.get(field)) for x in neg_calls]

bins = linspace(0, 150, 100)

plt.hist(x1, bins, alpha=0.6, color='b', label='POS')
plt.hist(x2, bins, alpha=0.3, color='r', label='NEG')
plt.show()

In [ ]:
list(itertools.chain.from_iterable([[calls.get(k)] for k in calls]))

In [ ]:
for k in sorted(calls.keys()):
    
    if calls.get(k):
        print('{}: total profit: {}, loss: {}, calls: {}, calls @ profit threshold: {}'.format(
            k, str(sum([x.get('profit') for x in calls[k] if 'profit' in x])), str(sum([x.get('profit') for x in calls[k] if 'profit' in x and x.get('profit') < 0])), 
            len(calls[k]), str(len([x for x in calls[k] if 'profit' in x and x.get('profit') > 0.05]))))

In [ ]:
mv = {'loss': [],
      'gain': []}

tv = {'loss': [],
      'gain': []}

tp = [0 for x in range(50)]

for k in sorted(calls.keys()):
    
    for e in calls.get(k):
        
        if e.get('profit') <= 0.02:
            mv['loss'].append(float(e.get('buy_volume')))
            tv['loss'].append(float(e.get('buy_volume_total')))
        else:
            mv['gain'].append(float(e.get('buy_volume')))
            tv['gain'].append(float(e.get('buy_volume_total')))
            
        tmp = floor(float(e.get('buy_volume_total')))

        tp[tmp] += float(e.get('profit'))
            
        print('{}: profit: {}, mean_volume: {}, total_volume: {}'.format(
            k, str(e.get('profit')), str(e.get('buy_volume')), str(e.get('buy_volume_total'))))

In [ ]:
calls.get('BTC-TIX')

In [ ]:
for ii in range(17, 29):

    orders = list(itertools.chain.from_iterable([[x for x in calls.get(y) if 0 < (x.get('time')-datetime(2017, 11, ii, 20, 14)).total_seconds() < 86400] 
                  for y in calls]))
    
    print('11/{}: {} orders, profit: {}'.format(str(ii), str(len(orders)), str(sum([x.get('profit') for x in orders if x.get('profit')]))))
    
    #pprint(sorted([(x.get('time'), x.get('profit')) for x in orders]))

In [ ]:
for x in sorted(calls.keys()):
    
    pprint((x, calls.get(x)))

In [ ]:
data = db.select_query('BTC-BAY', '*')

time, price, wprice, volume, buyvolume, sellvolume, buyorder, sellorder = zip(*data)

calls = call(time, wprice, buyvolume, buyorder,
                             duration=params.get('duration'),
                             price_lag_time=params.get('price_lag_time'),
                             price_lag_duration=params.get('price_lag_duration'),
                             price_lag_threshold=params.get('price_lag_threshold'),
                             volume_lag_duration=params.get('volume_lag_duration'),
                             volume_lag_threshold=params.get('volume_lag_threshold'),
                             profit_percent=params.get('profit_percent'),
                             stop_loss_percent=params.get('stop_loss_percent'),
                             stop_gain_percent=params.get('stop_gain_percent'),
                             max_hold_time=params.get('max_hold_time'),
                             wait_time=params.get('wait_time'))

In [ ]:
calls

In [ ]:
for table in tables:
    
    data = db.select_query(table, '*')

    time, price, wprice, volume, buyvolume, sellvolume, buyorder, sellorder = zip(*data)
    
    ind = [i for i in range(duration, len(buyvolume)) if mean(buyvolume[i-duration:i]) > 2 
           and variation(buyvolume[i-duration:i]) < 1
           and all([1 if x > 1 else 0 for x in buyvolume[i-duration:i]])]
    
    if ind:
        print('Found {} hits for {}: {}'.format(str(len(ind)), table, ','.join(map(str, ind))))
        plt.plot(range(len(time)), wprice, markevery=ind, marker='o', markerfacecolor='r', markersize=10)
        plt.show()
        plt.plot(range(len(time)), buyvolume, markevery=ind, marker='o', markerfacecolor='r', markersize=10)
        plt.show()
        for i in ind:
            print(i, buyvolume[i-duration:i])
            print(i, time[i])
        input('NEXT')
    else:
        print('No hits found for {}'.format(table))